In [19]:
pip install selenium


  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/b3/38/5aac23e57d61707f91914506902e621632de8dc56b60446459901469b9e2/selenium-4.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/1c/70/efa56ce2271c44a7f4f43533a0477e6854a0948e9f7b76491de1fd3be7c9/trio-0.26.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions~=4.9 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client~=1.8 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d1

In [21]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install requests-html

  Obtaining dependency information for requests-html from https://files.pythonhosted.org/packages/24/bc/a4380f09bab3a776182578ce6b2771e57259d0d4dbce178205779abdc347/requests_html-0.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyquery from https://files.pythonhosted.org/packages/76/f5/5067b48012967ea166b9bd0a015b69e0560e4c6e7c06f28d9bab8f9dd10b/pyquery-2.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for fake-useragent from https://files.pythonhosted.org/packages/e4/99/60d8cf1b26938c2e0a57e232f7f15641dfcd6f8deda454d73e4145910ff6/fake_useragent-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for parse from https://files.pythonhosted.org/packages/d0/31/ba45bf0b2aa7898d81cbbfac0e88c267befb59ad91a19e36e1bc5578ddb1/parse-1.20.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for bs4 from https://files.pythonhosted.org/packages/51/bb/bf7aab772a159614954d84aa832c129624ba6c32faa559dfb200a534e50b/bs4-0.0.2-py2.py3-non

In [ ]:
## Observasi HTML URL yang ingin discrapping

In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

# Inisialisasi driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [32]:
# Buka Page 1 URL
url = "https://search.kompas.com/search/?q=bpjs&submit=Submit#gsc.tab=0&gsc.q=bpjs&gsc.page=1"
driver.get(url)

In [33]:
# Tunggu beberapa detik agar konten JavaScript dimuat
time.sleep(5)

In [34]:
# Ambil konten halaman
page_source = driver.page_source

In [35]:
# Parse dengan BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [36]:
soup

<html class="js csstransitions" lang="en"><head>
<script async="" nonce="IOvmxAasolBKwB_qzaJVMg" src="https://sb.scorecardresearch.com/c2/8077308/cs.js" type="text/javascript"></script><script async="" nonce="IOvmxAasolBKwB_qzaJVMg" src="https://www.googletagmanager.com/gtag/js?id=G-77DJNQ0227&amp;cx=c&amp;_slc=1" type="text/javascript"></script><script async="" nonce="IOvmxAasolBKwB_qzaJVMg" src="//cse.google.com/adsense/search/async-ads.js"></script><script nonce="IOvmxAasolBKwB_qzaJVMg" src="https://securepubads.g.doubleclick.net/pagead/managed/js/gpt/m202409050101/pubads_impl_page_level_ads.js?cb=31086851"></script><script async="" src="//static.chartbeat.com/js/chartbeat.js" type="text/javascript"></script><script async="" nonce="IOvmxAasolBKwB_qzaJVMg" src="https://unpkg.com/web-vitals/dist/web-vitals.iife.js" type="text/javascript"></script><script async="" nonce="IOvmxAasolBKwB_qzaJVMg" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script 

In [ ]:
# Membuat dan mengeksekusi Fungsi Scrapping

In [23]:
# Inisialisasi driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Fungsi untuk scraping satu halaman
def scrape_page(url):
    driver.get(url)
    time.sleep(5)  # Tunggu konten JavaScript dimuat
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    target_div = soup.find('div', id='___gcse_0')
    scraped_data = []
    
    if target_div:
        for p in target_div.find_all("div", {"class": "gsc-thumbnail-inside"}):
            link_element = p.find("a", {"class": "gs-title"})
            link = link_element['href'] if link_element else ""
            title = link_element.get_text().strip() if link_element else ""
            
            date = ""
            if link:
                match = re.search(r'(\d{4}/\d{2}/\d{2})', link)
                if match:
                    date = match.group(1)
            
            scraped_data.append({
                'Judul': title,
                'Link': link,
                'Tanggal Upload': date
            })
    
    return scraped_data

# Inisialisasi DataFrame untuk menyimpan semua data
all_data = pd.DataFrame(columns=['Judul', 'Link', 'Tanggal Upload'])

# Loop melalui halaman 1 sampai 10
for page in range(1, 11):
    url = f"https://search.kompas.com/search/?q=bpjs&submit=Submit#gsc.tab=0&gsc.q=bpjs&gsc.page={page}"
    print(f"Scraping halaman {page}...")
    page_data = scrape_page(url)
    
    # Tambahkan data halaman ke DataFrame utama
    all_data = pd.concat([all_data, pd.DataFrame(page_data)], ignore_index=True)
    
    # Tunggu sebentar sebelum ke halaman berikutnya untuk menghindari pembatasan
    time.sleep(2)

# Tutup driver
driver.quit()

# Tampilkan hasil
print(all_data)

# Opsional: Simpan hasil ke file CSV
all_data.to_csv('scraping_kompas_bpjs.csv', index=False)

Scraping halaman 1...
Scraping halaman 2...
Scraping halaman 3...
Scraping halaman 4...
Scraping halaman 5...
Scraping halaman 6...
Scraping halaman 7...
Scraping halaman 8...
Scraping halaman 9...
Scraping halaman 10...
                                                Judul  \
0   Tak Mampu Bayar Iuran, Bisakah Menonaktifkan B...   
1            Cara Pindah Faskes BPJS Kesehatan Online   
2   Dirut BPJS: Dokter Asing Boleh Layani Pasien B...   
3   Sejahterakan Pekerja, BPJS Ketenagakerjaan Kuc...   
4   Tingkatkan Kualitas Layanan Faskes, BPJS Keseh...   
..                                                ...   
95  Syarat dan Cara Berobat di Luar Kota Pakai BPJ...   
96  BPJS Kesehatan Jadi Syarat Membuat dan Memperp...   
97  4 Jenis Kecelakaan yang Tidak Ditanggung BPJS ...   
98  BPJS Ketenagakerjaan dan Pemprov Jabar Hadirka...   
99  Tapera Tak Jamin Beri Rumah, Tak Bisa Disamaka...   

                                                 Link Tanggal Upload  
0   https://www.kompas.